## CHF Hospital Admissions

In [1]:
from sqlalchemy import create_engine
import re
import pandas as pd
from pprint import pprint
import psycopg2
from collections import Counter
import random

/Users/yonarp/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
%load_ext autoreload
%autoreload 2

Connecting to the Database

In [3]:
conn_string = "host='localhost' dbname='mimic' user='mimicuser' password='1234567890'"
conn = psycopg2.connect(conn_string)

Executing a Sample Query

In [ ]:
sql_query = """select * from mimic.mimiciii.noteevents"""
sql_query = re.sub(r'%', '%%', sql_query)
df = pd.read_sql_query(sql_query ,con=conn)
df.head()

In [5]:
NOTEEVENTS = df

DOUBTS: 
Which of the following description from above should I consider admission note:

In [6]:
print("Categories in the Notes ")
set(NOTEEVENTS['category'])

Categories in the Notes 


{'Case Management ',
 'Consult',
 'Discharge summary',
 'ECG',
 'Echo',
 'General',
 'Nursing',
 'Nursing/other',
 'Nutrition',
 'Pharmacy',
 'Physician ',
 'Radiology',
 'Rehab Services',
 'Respiratory ',
 'Social Work'}

In [61]:
print("\nMerging Category and Description and filtering Admission Notes")
temp = NOTEEVENTS['category'].map(str) + " " + NOTEEVENTS['description']
AdmissionNotes = [a for a in list(temp) if ("admission" in a.lower())]
print("Total Number of unique admission note types :", len(AdmissionNotes))
print("_"*80)
AdmissionNotes = Counter(AdmissionNotes)
i = 0;
print("\nPrinting counts of unique admission types and their counts in sorted order\n")
for key,value in AdmissionNotes.most_common():
    print('{0:65} ==> {1:10d}'.format(key, value))
    i += value
i


Merging Category and Description and filtering Admission Notes
Total Number of unique admission note types : 18632
________________________________________________________________________________

Printing counts of unique admission types and their counts in sorted order

Physician  Physician Resident Admission Note                      ==>      10652
Physician  Physician Attending Admission Note - MICU              ==>       3318
Physician  Physician Surgical Admission Note                      ==>       1109
Social Work Social Work Admission Note                            ==>       1106
Physician  Physician Attending Admission Note                     ==>        876
Physician  Physician Resident/Attending Admission Note - MICU     ==>        439
Physician  Physician Fellow / Attending Admission Note - MICU     ==>        147
Physician  MICU Attending Admission Note                          ==>         97
Physician  Physician Fellow/Attending Admission Note - MICU       ==>         

18632

In [7]:
print("Total Number of Hospital Admission Ids for Physician Resident Admission Note")
len(set(NOTEEVENTS[NOTEEVENTS['description'] == 'Physician Resident Admission Note']['hadm_id']))

Total Number of Hospital Admission Ids for Physician Resident Admission Note


5589

In [8]:
print("Total Number of Patients for whom description is Physician Resident Admission Note")
len(set(NOTEEVENTS[NOTEEVENTS['description'] == 'Physician Resident Admission Note']['subject_id']))

Total Number of Patients for whom description is Physician Resident Admission Note


4334

In [19]:
ADMISSION_NOTES = NOTEEVENTS[NOTEEVENTS['description'] == 'Physician Resident Admission Note']

In [18]:
ADMISSION_NOTES.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
276250,329153,25876,NaN,2189-08-04,NaT,2189-08-04 03:37:01,Physician,Physician Resident Admission Note,16117.0,None,Chief Complaint: Lethargy\n HPI:\n 39 yea...
284359,318124,11486,168769.0,2198-02-02,2198-02-02 07:40:00,2198-02-02 07:40:29,Physician,Physician Resident Admission Note,19858.0,None,Chief Complaint: Transferred s/p vertebroplas...
285771,317203,28973,158953.0,2159-02-21,2159-02-21 00:29:00,2159-02-21 00:29:17,Physician,Physician Resident Admission Note,16671.0,None,Chief Complaint: Atrial fibrillation with RVR...
287760,318265,4588,186721.0,2113-03-04,2113-03-04 05:42:00,2113-03-04 09:16:35,Physician,Physician Resident Admission Note,14180.0,None,Chief Complaint: Cough/SOB & hyperglycemia\n ...
288337,318273,4588,186721.0,2113-03-04,2113-03-04 05:42:00,2113-03-04 10:40:26,Physician,Physician Resident Admission Note,14180.0,None,Chief Complaint: Cough/SOB & hyperglycemia\n ...


In [16]:
ADMISSION_NOTES.shape

(10654, 11)

In [89]:
ADMISSION_NOTES.to_csv("ADMISSION_NOTES.csv", columns=['row_id','subject_id', 'hadm_id'], index=False)

### Searching the Text Field for the following keywords: <br/><br/>
o	HFrEF<br/>
o	heart failure with reduced ejection<br/>
o	heart failure with preserved ejection fraction<br/> 
o	HFpEF<br/>
o	Congestive heart failure<br/>
o	diastolic heart failure<br/>
o	systolic heart failure <br/>
o	CHF<br/>
<br/><br/>
These Keywords tell us that the Hospital staff had a bias that the Patient had CHF
<br/>
<br/>
`Other Keywords Identified which are Typical of CHF`
- preserved LVEF
- hepatic congestion from heart failure

In [20]:
searchString = """Congestive heart failure|
                heart failure with reduced ejection|
                heart failure with preserved ejection fraction|
                HFpEF|
                HFrEF|
                diastolic heart failure|
                systolic heart failure|
                CHF"""
t = ""
for searchWord in searchString.split('\n'):
    t += searchWord.strip() 
searchString = t
CHF_NOTES = ADMISSION_NOTES[ADMISSION_NOTES['text'].str.contains(searchString, flags=re.IGNORECASE, regex=True)]

In [60]:
# Printing CHF Notes to a CSV
CHF_NOTES.to_csv("CHF_NOTES.csv", columns=['subject_id', 'hadm_id'])

In [62]:
pd.read_csv("CHF_NOTES.csv").head()

,Unnamed: 0,subject_id,hadm_id
0,284359,11486,168769.0
1,295822,22520,139893.0
2,299310,29765,173513.0
3,307475,28915,168883.0
4,308887,29705,161764.0


### Testing 5 Random Samples

In [218]:
random.seed(10)
rowIndex = random.choice(list(range(CHF_NOTES.shape[0])))
print("rowIndex", rowIndex)
print("HADM_ID", CHF_NOTES.iloc[rowIndex]["hadm_id"])
print(CHF_NOTES.iloc[rowIndex]["text"])

rowIndex 2340
HADM_ID 127310.0
Chief Complaint:  PCP:  [**First Name4 (NamePattern1) 326**] [**Last Name (NamePattern1) 4679**].
   .
   CHIEF COMPLAINT:  abdominal distension.
   REASON FOR MICU ADMISSION: chronic trach requires suctioning.
   HPI:
   Pt is a 43 y.o male with h.o anoxic brain injury (non-verbal) s/p
   cardiac arrest [**9-10**] (EF 20%), chronic trach (requires frequent
   suctioning), who was found to have abdominal distention, fluid
   retention and pitting edema in his upper and lower extremities, fever,
   ARF, abnormal LFTs at [**Hospital 258**] rehab. He is on Vanco ([**1-1**]), levaquin
   (+ucx [**1-1**]). He was noted to have a leukocytosis. [**12-6**] bcx bottles
   growing staph aureus.
   .
   Pt was recently admitted [**Date range (1) 11515**] with concerns of bleeding from a
   trach, formerly admitted for tongue laceration ([**Date range (1) 11516**]). During
   that course pt had all of his teeth removed and was clonazepam. He was
   noted to have a hi

In [219]:
random.seed(20)
rowIndex = random.choice(list(range(CHF_NOTES.shape[0])))
print("rowIndex", rowIndex)
print("HADM_ID", CHF_NOTES.iloc[rowIndex]["hadm_id"])
print(CHF_NOTES.iloc[rowIndex]["text"])

rowIndex 2960
HADM_ID 123366.0
Chief Complaint:  Chest Pain
   HPI:
   64 y/o gentleman with hypertension, diabetes mellitus, dyslipidemia,
   PAF woke up this morning around 3:45 AM with severe SSCP with radiation
   to back.  Patient had associated diaphoresis, N and vomitting.  He
   walked across to [**Hospital1 19**] [**Location (un) 250**] and was found to be in anterior STEMI.
   He was given ASA 324mg, Plavix 600mg, Integrillin bolus, Heparin 7500u
   and a sl nitro with no change in symptom.  He was transferred to
   [**Hospital1 19**]-[**Location (un) 23**] cath lab for emergent cardiac catheterization.  On arrival
   he was awake and alert, and chest pain free.  Integrillin gtt was
   started.
   .
   During cardiac catheterization he was found to have mid LAD lesion and
   recieved Xience x 2.  LV gram showed LVEF of 45% with moderate
   anterolateral hypokinesis.  His angioseal was unsuccessful and was
   placed on clamp.  Patient was transfered to CCU for further manageme

In [220]:
random.seed(30)
rowIndex = random.choice(list(range(CHF_NOTES.shape[0])))
print("rowIndex", rowIndex)
print("HADM_ID", CHF_NOTES.iloc[rowIndex]["hadm_id"])
print(CHF_NOTES.iloc[rowIndex]["text"])

rowIndex 2208
HADM_ID 197499.0
Chief Complaint:  dyspnea, volume overload
   HPI:
   Mr. [**Known lastname **] is a 68 yo M with Ischemic cardiomyopathy EF < 20%, s/p CABG
   x2, s/p BIV ICD admitted with acute on chronic CHF exacerbation
   admitted for diuresis.  He reports that over the past two months he has
   had progressive dyspnea on exertion to the point that he is now able to
   ambulate only about one block on a flat surface.  He also reports
   increase in abdominal girth.  His weight is 257 pounds with a dry
   weight of 210-220 pounds.  Otherwise he reports that he has been
   feeling well and denies fevers, chills chest pain, PND, orthopnea,
   dyspnea at rest, nausea, vomiting, abdominal pain, diarrhea,
   constipation, hematochezia, melena.  He does endores imbalance with
   walking.
   .
   Cardiac review of systems is notable for absence of chest pain, dyspnea
   on exertion, paroxysmal nocturnal dyspnea, orthopnea, ankle edema,
   palpitations, syncope or presyncope

In [221]:
random.seed(40)
rowIndex = random.choice(list(range(CHF_NOTES.shape[0])))
print("rowIndex", rowIndex)
print("HADM_ID", CHF_NOTES.iloc[rowIndex]["hadm_id"])
print(CHF_NOTES.iloc[rowIndex]["text"])

rowIndex 1878
HADM_ID 121377.0
Chief Complaint:  dysphagia and regurgitation
   HPI:
   85M with CAD, CHF EF 20%, VT s/p ICD, ESRD on HD, p/w dysphagia. He ate
   a meal on [**11-16**] that included [**Last Name (un) 7211**]. Following that meal he felt the
   sensation of food stuck in throat and regurgitated food. He has been
   unable to take down and keep down any food or fluid since that meal.
   Everything that he has eaten gets regurgitated up slowly. He denies any
   chest pain currently but had some mild discomfort one day prior to
   admission. He denies any difficulty managing his secretions, denies
   drooling.  He lives with his daughter. The family initially watched the
   symptoms hoping that they would resolve, however the symptoms continued
   and he was brought to the ED. He denies f/c. No nausea. No coughing or
   SOB. In the ED, GI was contact[**Name (NI) **]. CXR showed no PNA, metoclopramide was
   given x1. He was admitted to hospitalist service for further manag

In [222]:
random.seed(50)
rowIndex = random.choice(list(range(CHF_NOTES.shape[0])))
print("rowIndex : ", rowIndex)
print("HADM_ID", CHF_NOTES.iloc[rowIndex]["hadm_id"])
print(CHF_NOTES.iloc[rowIndex]["text"])

rowIndex :  2037
HADM_ID 163603.0
Chief Complaint:  hypotension and unresponsiveness
   HPI:
   68 F with history of large B cell NHL s/p autoBMT, history of
   intraparencymal hemorrhage and SDH s/p evacution (at baseline aphasic),
   seizure, HIT, DVT/PE s/p IVC filter; transfer from neurology service
   for episode of unresponsiveness and hypotension.  Patient has been
   undergoing recent workup with multiple admits for episodic
   unresponsiveness.  Initially admitted to East on [**2160-3-22**] after
   presenting to the ED with functional decline at home, characterized by
   increased episodes of agitation and yelling.   On the medicine floor
   had episode of unresponsiveness and given ativan given with question of
   underlying seizure as a cause with improvement of symtoms; eventual
   transfer to [**Hospital Unit Name 44**] after this episode.  Then called out to neuro floor
   for continuous EEG monitoring.  On the neurology service, she has been
   monitored with continuous

### Creating a Hospital Admission Unique IDs list for Patients with CHF at Admission

In [28]:
chf_hadm_ids_based_on_admission_notes = [element for element in set(list(CHF_NOTES['hadm_id'])) if str(element) != 'nan']

## Selecting Patients with CHF as major diagnosis at Admission based on Admission Table

In [169]:
sql_query = """select * from mimic.mimiciii.admissions"""
sql_query = re.sub(r'%', '%%', sql_query)
df = pd.read_sql_query(sql_query ,con=conn)
df.head()
ADMISSIONS = df

In [172]:
ADMISSIONS.shape

(58976, 19)

In [171]:
ADMISSIONS.head()

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,None,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,None,CATHOLIC,MARRIED,WHITE,NaT,NaT,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,None,PROTESTANT QUAKER,SINGLE,WHITE,NaT,NaT,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,None,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [176]:
CHF_DIAGNOSIS = ADMISSIONS[ADMISSIONS['diagnosis'].str.contains("Congestive heart failure", flags=re.IGNORECASE, regex=True,  na=False)]

In [29]:
chf_notes = set(chf_hadm_ids_based_on_admission_notes)
chf_dianogsis = set([element for element in set(list(CHF_DIAGNOSIS['hadm_id'])) if str(element) != 'nan'])
print(len(chf_notes))
print(len(chf_dianogsis))
chf_total = chf_notes.union(chf_dianogsis)
print("Total Number of Patients ", len(chf_total))

NameError: name 'CHF_DIAGNOSIS' is not defined

In [188]:
searchString = """Congestive heart failure|
                heart failure with reduced ejection|
                heart failure with preserved ejection fraction|
                HFpEF|
                HFrEF|
                diastolic heart failure|
                systolic heart failure|
                CHF"""
t = ""
for searchWord in searchString.split('\n'):
    t += searchWord.strip() 
searchString = t
temp = ADMISSIONS[ADMISSIONS['diagnosis'].str.contains(searchString, flags=re.IGNORECASE, regex=True, na=False)]

In [189]:
temp = set([element for element in set(list(temp['hadm_id'])) if str(element) != 'nan'])

chf_notes

In [198]:
ADMISSIONS[ADMISSIONS['hadm_id'] == 100522]

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
22027,23157,18982,100522,2139-07-22 06:56:00,2139-08-06 12:00:00,2139-08-06 12:00:00,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,Medicare,ENGL,PROTESTANT QUAKER,SEPARATED,BLACK/AFRICAN AMERICAN,2139-07-22 03:12:00,2139-07-22 08:30:00,BOWEL OBSTRUCTION,1,1


In [208]:
print(ADMISSION_NOTES[ADMISSION_NOTES['hadm_id'] == 100522]["text"].iloc[0])

Chief Complaint:  Altered mental status, abdominal pain
   HPI:
   Ms. [**Known lastname 1161**] is a 69 year old female with history of DM, COPD, ventilator
   dependent, hypertension, rectus sheath hematoma [**6-/2139**] who was brought
   to [**Hospital 120**] Hospital with altered mental status and abdominal pain.
   Abdominal CT was done at outside hospital which showed partial SBO.  CT
   head at OSH was negative for intracranial process.  She was transferred
   to [**Hospital1 54**] for further work up given her recent admission here.
   .
   In our ED, vital signs were BP 115/50, HR 90, RR 16, O2 sat 100% on
   trach collar.  Labs notable for positive UA, WBC count 12.9 (73%
   neutrophils), creatinine 6.2 up from last d/c 4.2, troponin 1.51 (CKMB
   normal), hct 30.2 (up from b/l of 24-25 last admission).  Blood and
   urine cultures sent from ED.  She was given 1L NS, Cipro 400mg x1,
   Aspirin 600mg PR, Tylenol 1g.  She was seen by surgery for evaluation
   of partial SBO.  

DOUBTS:


* What if there is NO in the note

* Social history


## Investigate Echo notes of patients with a history of CHF prior to ICU admission and see if they fit the CHF echo note criterion described in supplemental table (attached below!)

In [214]:
ECHONOTES = NOTEEVENTS[NOTEEVENTS.category == 'Echo']

In [223]:
print(ECHONOTES[ECHONOTES['hadm_id'] == 127310]["text"].iloc[0])

PATIENT/TEST INFORMATION:
Indication: Endocarditis. Cardiomyopathy.
Height: (in) 70
Weight (lb): 200
BSA (m2): 2.09 m2
BP (mm Hg): 110/70
HR (bpm): 105
Status: Inpatient
Date/Time: [**2203-1-7**] at 17:25
Test: Portable TEE (Complete)
Doppler: Full Doppler and color Doppler
Contrast: None
Technical Quality: Adequate


INTERPRETATION:

Findings:

The descending thoracic aorta was not well visualized due to poor contact.
LEFT ATRIUM: Mild LA enlargement. No spontaneous echo contrast or thrombus in
the LA/LAA or the RA/RAA. Good (>20 cm/s) LAA ejection velocity. All four
pulmonary veins identified and enter the left atrium.

RIGHT ATRIUM/INTERATRIAL SEPTUM: A catheter or pacing wire is seen in the RA
and extending into the RV. No ASD by 2D or color Doppler.

LEFT VENTRICLE: Severe global LV hypokinesis.

RIGHT VENTRICLE: Severe global RV free wall hypokinesis.

AORTIC VALVE: Normal aortic valve leaflets (3). No masses or vegetations on
aortic valve. No AR.

MITRAL VALVE: Normal mitral val

In [224]:
print(ECHONOTES[ECHONOTES['hadm_id'] == 123366]["text"].iloc[0])

PATIENT/TEST INFORMATION:
Indication: Myocardial infarction.
Height: (in) 74
Weight (lb): 289
BSA (m2): 2.54 m2
BP (mm Hg): 113/65
HR (bpm): 81
Status: Inpatient
Date/Time: [**2118-10-4**] at 10:00
Test: Portable TTE (Complete)
Doppler: Full Doppler and color Doppler
Contrast: None
Technical Quality: Suboptimal


INTERPRETATION:

Findings:

LEFT ATRIUM: Mild LA enlargement.

RIGHT ATRIUM/INTERATRIAL SEPTUM: Normal RA size.

LEFT VENTRICLE: Mild symmetric LVH. Normal LV cavity size. Mild regional LV
systolic dysfunction. No resting LVOT gradient. Moderate resting LVOT
gradient.

LV WALL MOTION: Regional LV wall motion abnormalities include: basal anterior
- hypo; mid anterior - akinetic; mid anteroseptal - akinetic; anterior apex -
akinetic;

RIGHT VENTRICLE: Normal RV chamber size and free wall motion.

AORTA: Mildly dilated aortic sinus. Normal ascending aorta diameter. Mildly
dilated aortic arch. No 2D or Doppler evidence of distal arch coarctation.

AORTIC VALVE: Normal aortic valve

In [225]:
print(ECHONOTES[ECHONOTES['hadm_id'] == 197499]["text"].iloc[0])

PATIENT/TEST INFORMATION:
Indication: Left ventricular function. Prosthetic valve function.
Height: (in) 71
Weight (lb): 255
BSA (m2): 2.34 m2
BP (mm Hg): 128/109
HR (bpm): 60
Status: Inpatient
Date/Time: [**2179-7-30**] at 09:58
Test: Portable TTE (Complete)
Doppler: Full Doppler and color Doppler
Contrast: Definity
Technical Quality: Adequate


INTERPRETATION:

Findings:

This study was compared to the prior study of [**2178-3-2**].


LEFT ATRIUM: Moderate LA enlargement.

RIGHT ATRIUM/INTERATRIAL SEPTUM: Moderately dilated RA. A catheter or pacing
wire is seen in the RA and extending into the RV.

LEFT VENTRICLE: Normal LV wall thickness. Severely dilated LV cavity. Severe
regional LV systolic dysfunction. No LV mass/thrombus.

RIGHT VENTRICLE: Mildly dilated RV cavity. Borderline normal RV systolic
function.

AORTA: Normal diameter of aorta at the sinus, ascending and arch levels.

AORTIC VALVE: Moderately thickened aortic valve leaflets.

MITRAL VALVE: Bioprosthetic mitral valve p

In [ ]:
print(ECHONOTES[ECHONOTES['hadm_id'] == 121377]["text"].iloc[0])

## CHF with SEPSIS 

From MARTA Given Script

In [41]:
SEPSIS = pd.read_csv("./sepsis_chf.csv")
SEPSIS.head()

,'subject_id','hadm_id','angus','chd','chd_class'
0,22,165315,0,0,''
1,23,152223,0,0,''
2,23,124321,0,0,''
3,24,161859,0,0,''
4,25,129635,0,0,''


In [42]:
sepsis_hadm_ids = [element for element in set(list(SEPSIS['\'hadm_id\''])) if str(element) != 'nan']

In [70]:
_SEPSIS = SEPSIS[SEPSIS['\'angus\''] == 1]

In [71]:
_sepsis_hadm_ids = [element for element in set(list(_SEPSIS['\'hadm_id\''])) if str(element) != 'nan']

In [73]:
len(_sepsis_hadm_ids)

15254

In [72]:
len(set(_sepsis_hadm_ids).intersection(set(chf_hadm_ids_based_on_admission_notes)))

792

In [63]:
len(sepsis_hadm_ids)

58976

In [53]:
# 180367.0, 180629.0, 197146.0,  164522.0, 125506.0
SEPSIS[SEPSIS['\'hadm_id\''] == 125506]

,'subject_id','hadm_id','angus','chd','chd_class'
48721,58515,125506,1,1,'UNSPECIFIED'


In [77]:
# NUmber of Patients in the Angus Script who have admission notes 


admission_notes_hadm_ids = set([element for element in set(list(ADMISSION_NOTES['hadm_id'])) if str(element) != 'nan'])

In [82]:
# admission_notes_hadm_ids = [int(admission_notes_hadm_ids) for element in admission_notes_hadm_ids]
t = set()
for element in admission_notes_hadm_ids:
    t.add(int(element))

{180224, 114690, 114691, 131078, 147462, 122892, 147469, 147471, 106512, 188434, 196627, 139284, 163861, 172052, 139286, 114707, 180246, 122900, 188441, 163870, 106527, 122911, 106530, 114726, 131114, 131118, 172079, 114735, 122926, 172082, 188468, 139318, 172088, 163902, 114752, 196673, 139329, 172101, 172102, 188488, 163913, 114761, 131148, 139340, 163918, 131151, 106576, 114769, 139346, 155726, 155727, 155731, 122966, 131162, 180314, 131164, 122972, 196702, 196712, 172139, 114795, 122988, 163951, 131184, 196721, 131186, 106610, 188536, 196729, 172162, 172163, 106628, 106630, 196744, 131209, 163976, 147595, 188553, 196749, 114830, 180367, 114832, 188565, 114838, 123031, 163992, 139416, 180377, 114843, 131235, 164005, 180391, 188583, 180393, 196778, 180395, 155819, 114861, 196782, 147630, 188587, 106673, 106674, 180402, 114867, 164021, 180407, 139449, 155834, 180411, 188604, 180414, 106688, 180416, 188612, 131271, 196808, 196809, 172232, 172233, 114887, 196813, 147659, 123083, 164048,

In [84]:
len(t.intersection(_sepsis_hadm_ids))

2147

In [99]:
SEPSIS_WITH_ADMISSION_NOTES = ADMISSION_NOTES[ADMISSION_NOTES['hadm_id'].isin(_sepsis_hadm_ids)]

In [100]:
SEPSIS_WITH_ADMISSION_NOTES.head()

,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
285771,317203,28973,158953.0,2159-02-21,2159-02-21 00:29:00,2159-02-21 00:29:17,Physician,Physician Resident Admission Note,16671.0,None,Chief Complaint: Atrial fibrillation with RVR...
299310,320266,29765,173513.0,2167-02-14,2167-02-14 03:26:00,2167-02-14 07:26:54,Physician,Physician Resident Admission Note,16947.0,None,"Chief Complaint: dyspnea, hypoxic resp failur..."
308887,316300,29705,161764.0,2126-02-09,2126-02-09 23:38:00,2126-02-09 23:38:15,Physician,Physician Resident Admission Note,16198.0,None,Chief Complaint: respiratory distress\n HPI...
308888,316301,29705,161764.0,2126-02-09,2126-02-09 23:38:00,2126-02-09 23:43:14,Physician,Physician Resident Admission Note,16198.0,None,Chief Complaint: respiratory distress\n HPI...
308889,316302,29705,161764.0,2126-02-09,2126-02-09 23:38:00,2126-02-09 23:48:01,Physician,Physician Resident Admission Note,16198.0,None,Chief Complaint: respiratory distress\n HPI...


In [102]:
SEPSIS_WITH_ADMISSION_NOTES.shape

(4506, 11)

In [101]:
SEPSIS_WITH_ADMISSION_NOTES.to_csv("SEPSIS_WITH_ADMISSION_NOTES.csv", columns=['row_id','subject_id', 'hadm_id'], index=False)

In [103]:
len(set(SEPSIS_WITH_ADMISSION_NOTES['hadm_id']))

2147

In [109]:
CHF_SEPSIS_NOTES.shape

(1567, 11)

In [107]:
CHF_SEPSIS_NOTES = CHF_NOTES[CHF_NOTES['hadm_id'].isin(_sepsis_hadm_ids)]

In [108]:
CHF_SEPSIS_NOTES.to_csv("CHF_SEPSIS_NOTES.csv", columns=['row_id','subject_id', 'hadm_id'], index=False)